In [19]:
no_acc = [44, 2, 4, 3, 3, 23, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 43, 0, 0, 0, 0, 19, 13, 4, 0, 0, 0, 10, 6, 7, 46, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 6, 0, 0, 0, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 19, 0, 9, 4, 0, 25, 4, 5, 0, 0, 0, 35, 0, 0, 0, 0, 27, 0, 0, 0, 0, 0, 0, 4, 1, 17, 0, 6, 0, 0, 15, 0, 11, 0, 0, 12, 0, 1, 0, 0, 7, 0, 0, 7, 0, 0, 0, 0, 12, 0, 2, 0, 0, 2, 8, 5, 0, 3, 0, 6, 10, 0, 0, 0, 0, 0, -1, -1, -128, -128, -128, -128, -128, -128, -128, 127, -128, -128, -128, -128]
for x in range(0,3):
    print(no_acc[64*x])
    

44
0
-128


In [64]:
import torch
import torch.nn as nn

# Define the input tensor (from `din` in Rust code) with shape [N, C_in, H, W]
input_tensor = torch.tensor([
    [
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5]
    ],
    [
        [5, 4, 3, 2, 1],
        [5, 4, 3, 2, 1],
        [5, 4, 3, 2, 1],
        [5, 4, 3, 2, 1],
        [5, 4, 3, 2, 1]
    ],
    [
        [3, 2, 1, 4, 5],
        [3, 2, 1, 4, 5],
        [3, 2, 1, 4, 5],
        [3, 2, 1, 4, 5],
        [3, 2, 1, 4, 5]
    ], 
    [
        [2, 4, 5, 1, 3],
        [2, 4, 5, 1, 3],
        [2, 4, 5, 1, 3],
        [2, 4, 5, 1, 3],
        [2, 4, 5, 1, 3],
    ]
], dtype=torch.int8).unsqueeze(0).float()  # Shape: [1, 3, 5, 5]

# Define the Conv2d layer with groups (4 groups for 4 channels in the example)
conv_group_2 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, groups=1)
conv_group_manual = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, groups=1)

# Convert Rust `wgt` to PyTorch tensor and reshape for the conv layers
wgt = torch.tensor([
    [[1,2,3], [4,5,6], [7,8,9]],
    [[9,8,7], [4,5,6], [1,2,3]],
    [[1,4,9], [2,5,8], [3,6,7]],
    [[9,4,1], [8,5,2], [7,6,3]],
    
    [[1,2,3], [4,5,6], [7,8,9]],
    [[9,8,7], [4,5,6], [1,2,3]],
    [[1,4,9], [2,5,8], [3,6,7]],
    [[9,4,1], [8,5,2], [7,6,3]]
], dtype=torch.int8).reshape(2, 4, 3, 3).float()  # Shape: [8, 3, 3, 3]

# Convert Rust `bias` to PyTorch tensor
bias = torch.tensor([-16, 16, 0, 0, 8, -8, -36, 36], dtype=torch.int16).float()

# Apply weights and bias to the conv_group_2 layer
conv_group_2.weight = nn.Parameter(wgt)
conv_group_2.bias = nn.Parameter(bias)

# Manually implemented version with separate conv layers and concatenation
class ManualGroupedConv(nn.Module):
    def __init__(self, weights, biases):
        super(ManualGroupedConv, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3)
        
        # Initialize weights and biases
        self.conv1.weight = nn.Parameter(weights[0])
        self.conv2.weight = nn.Parameter(weights[1])
        self.conv3.weight = nn.Parameter(weights[2])
        self.conv4.weight = nn.Parameter(weights[3])
        self.conv1.bias = nn.Parameter(biases[0])
        self.conv2.bias = nn.Parameter(biases[1])
        self.conv3.bias = nn.Parameter(biases[2])
        self.conv4.bias = nn.Parameter(biases[3])

    def forward(self, x):
        splits = torch.split(x, 1, dim=1)  # Split channels
        out1 = self.conv1(splits[0])
        out2 = self.conv2(splits[1])
        out3 = self.conv3(splits[2])
        out4 = self.conv4(splits[3])
        return torch.cat([out1, out2, out3, out4], dim=1)  # Concatenate the outputs

# Create manual group conv with weights and biases
weights = wgt.chunk(4, dim=1)
for w in weights:
    print("____________________________________")
    print(w)
biases = bias.chunk(4)
manual_group_conv = ManualGroupedConv(weights, biases)

# Apply the layers to the input tensor
output_manual = manual_group_conv(input_tensor)

# Compare the outputs
print(f"Output with manually implemented groups: {output_manual.shape}")

# Print outputs
print("Output with manually implemented groups:\n", output_manual)


____________________________________
tensor([[[[1., 2., 3.],
          [4., 5., 6.],
          [7., 8., 9.]]],


        [[[1., 2., 3.],
          [4., 5., 6.],
          [7., 8., 9.]]]])
____________________________________
tensor([[[[9., 8., 7.],
          [4., 5., 6.],
          [1., 2., 3.]]],


        [[[9., 8., 7.],
          [4., 5., 6.],
          [1., 2., 3.]]]])
____________________________________
tensor([[[[1., 4., 9.],
          [2., 5., 8.],
          [3., 6., 7.]]],


        [[[1., 4., 9.],
          [2., 5., 8.],
          [3., 6., 7.]]]])
____________________________________
tensor([[[[9., 4., 1.],
          [8., 5., 2.],
          [7., 6., 3.]]],


        [[[9., 4., 1.],
          [8., 5., 2.],
          [7., 6., 3.]]]])
Output with manually implemented groups: torch.Size([1, 8, 3, 3])
Output with manually implemented groups:
 tensor([[[[ 80., 125., 170.],
          [ 80., 125., 170.],
          [ 80., 125., 170.]],

         [[112., 157., 202.],
          [112., 1

In [32]:
import torch
import torch.nn as nn
import copy

# Define the input tensor
input_tensor = torch.randn(1, 4, 5, 5)  # Example input with 4 channels

# Define the Conv2d layer with groups
conv_group = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, groups=4)
conv_group_2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, groups=1)
conv_group_manual = copy.deepcopy(conv_group)  # Ensure they start with the same weights and biases

# Manually implemented version with separate conv layers and concatenation
class ManualGroupedConv(nn.Module):
    def __init__(self, weights, biases):
        super(ManualGroupedConv, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3)
        
        # Initialize weights and biases
        self.conv1.weight = nn.Parameter(weights[0])
        self.conv2.weight = nn.Parameter(weights[1])
        self.conv3.weight = nn.Parameter(weights[2])
        self.conv4.weight = nn.Parameter(weights[3])
        self.conv1.bias = nn.Parameter(biases[0])
        self.conv2.bias = nn.Parameter(biases[1])
        self.conv3.bias = nn.Parameter(biases[2])
        self.conv4.bias = nn.Parameter(biases[3])

    def forward(self, x):
        splits = torch.split(x, 1, dim=1)  # Split channels
        out1 = self.conv1(splits[0])
        out2 = self.conv2(splits[1])
        out3 = self.conv3(splits[2])
        out4 = self.conv4(splits[3])
        return torch.cat([out1, out2, out3, out4], dim=1)  # Concatenate the outputs

# Extract weights and biases from grouped Conv2d layer
weights = conv_group.weight.data.chunk(4, dim=0)
biases = conv_group.bias.data.chunk(4, dim=0)

# Create manually implemented group conv layer with initialized weights and biases
manual_group_conv = ManualGroupedConv(weights, biases)

# Apply the layers to the input tensor
#output_group = conv_group_manual(input_tensor)
output_group = conv_group_2(input_tensor)
output_manual = manual_group_conv(input_tensor)


print(weights)
print(biases)

# Compare the outputs
print(f"Output with Conv2d groups: {output_group.shape}")
print(f"Output with manually implemented groups: {output_manual.shape}")
print(f"Outputs are equal: {torch.allclose(output_group, output_manual)}")
for x in output_group:
    print(x)
print(output_group)
print(output_manual)
# In_channels = in_channels / groups
# Out_channels = out_channels / groups


(tensor([[[[-0.1996, -0.0541, -0.0578],
          [ 0.3317, -0.0094,  0.1468],
          [-0.0404, -0.1715, -0.2873]]],


        [[[-0.1923, -0.2847,  0.2676],
          [ 0.0068, -0.3035, -0.0364],
          [-0.2271,  0.3301, -0.1119]]]]), tensor([[[[-0.1875,  0.0898,  0.3057],
          [ 0.1733,  0.2023, -0.2250],
          [-0.1055,  0.2657, -0.2829]]],


        [[[ 0.2047,  0.1589,  0.2478],
          [ 0.2174,  0.2594, -0.2384],
          [ 0.1749,  0.0625,  0.0872]]]]), tensor([[[[ 0.0547,  0.2717,  0.2748],
          [ 0.3173,  0.2304,  0.3139],
          [-0.0540, -0.1187, -0.0311]]],


        [[[ 0.0020,  0.1235,  0.0810],
          [-0.0303,  0.3200,  0.1364],
          [ 0.3146,  0.0158,  0.3014]]]]), tensor([[[[ 0.2170, -0.2991,  0.0782],
          [ 0.3103,  0.0825, -0.0578],
          [ 0.0687,  0.2704,  0.2271]]],


        [[[ 0.3098, -0.0075, -0.2496],
          [-0.3150,  0.2385, -0.1995],
          [-0.2754, -0.2036,  0.0391]]]]))
(tensor([-0.2930,  0.1524]), te